<a href="https://colab.research.google.com/github/H-jung/LPOINT_Contest/blob/main/%EB%A1%AF%EB%8D%B0%EB%A9%A4%EB%B2%84%EC%8A%A4_%EC%A0%9C%ED%9C%B4%EC%82%AC_%EC%9C%A0%ED%86%B5%EC%82%AC_%EB%B9%84%EC%9C%A8_nolpay.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 롯데멤버스 빅데이터경진대회
- 팀장 : 임현정
- 팀원 : 하혜영, 김윤지

## 사용 라이브러리

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import matplotlib
matplotlib.rc('font',family = 'Malgun Gothic')
matplotlib.rc('axes',unicode_minus = False)

import warnings
warnings.filterwarnings('ignore')

In [4]:
nolpay = pd.read_csv("/content/drive/MyDrive/백업파일/공모전/LPOINT_BIG_COMP/nolpay2.csv",parse_dates=['구매일자'])

In [111]:
# 유통사 상품 구매 내역
data2 = pd.read_csv("/content/drive/MyDrive/백업파일/공모전/LPOINT_BIG_COMP/LPOINT_BIG_COMP_02_PDDE.csv",parse_dates = ['de_dt'])
data2.columns = ['고객번호','영수증번호','채널','제휴사','점포코드','상품코드','구매일자','구매시간','구매금액','구매수량']
data2['채널'] = np.where(data2['채널'] == 1, "오프라인","온라인")

# 유통사 상품 카테고리 마스터 수정
data4 = pd.read_csv("/content/drive/MyDrive/백업파일/공모전/LPOINT_BIG_COMP/data4_new.csv");data4.head()

,상품코드,소분류,대분류,중분류,상품구분
0,PD0001,소파,가구,거실가구,생활
1,PD0002,스툴/리빙의자,가구,거실가구,생활
2,PD0003,탁자,가구,거실가구,생활
3,PD0004,장식장/진열장,가구,거실가구,생활
4,PD0005,기타가구,가구,기타가구,생활


### 고객별 선호 제휴사구분
: 유통사 16993명 / 제휴사 3975명

In [96]:
count_rcp = nolpay.groupby(['고객번호','제휴사구분'])[['영수증번호']].nunique().reset_index()
cop_cust = count_rcp.groupby('고객번호')[['영수증번호']].agg(lambda x : count_rcp['제휴사구분'][x.idxmax()]).reset_index()

In [109]:
cust_A = cop_cust[cop_cust.영수증번호=="유통사"].고객번호.tolist();print(len(cust_A))
cust_BE = cop_cust[cop_cust.영수증번호=="제휴사"].고객번호.tolist();print(len(cust_BE))

16993
3975


#### 1) 유통사의 비율이 높은 사람들이 어떤 물품을 많이 사는지?

In [120]:
data2[data2.고객번호.isin(cust_A)].merge(data4).groupby('고객번호')[['상품구분']].agg(lambda x:x.value_counts().idxmax()).reset_index()

,고객번호,상품구분
0,M000034966,식품
1,M000201112,식품
2,M000225114,식품
3,M000261625,건강/뷰티
4,M000350564,의류
...,...,...
16988,M999468256,식품
16989,M999492154,의류
16990,M999515910,식품
16991,M999673157,기타


#### 2) 제휴사의 비율이 높은 사람들이 어떤 제휴사를 많이 사용하는지?

In [108]:
t1 = nolpay.loc[nolpay.고객번호.isin(cust_BE)].groupby(['고객번호','제휴사'])['영수증번호'].nunique().reset_index()
t1.groupby('고객번호')[['영수증번호']].agg(lambda x : t1['제휴사'][x.idxmax()]).reset_index()

,고객번호,영수증번호
0,M000059535,C01
1,M000608261,C02
2,M000803099,D02
3,M000965026,C01
4,M001080017,A01
...,...,...
3970,M999181273,A04
3971,M999428656,D01
3972,M999493501,D02
3973,M999708287,C01
